# Web Scraping: Selenium
_Automate your browser._ <br>
_Collect data from dynamically generated web pages or those requiring user interaction._

### Docs

- [Selenium homepage](https://www.seleniumhq.org/) 
- [Selenium documentation](https://selenium-python.readthedocs.io/) - unofficial, but helpful

### Installation

With conda:
- `conda install -c conda-forge selenium`

With pip:
- `pip install -U selenium`

#### ChromeDriver

You will also need to install a web driver to use Selenium.  ChromeDriver is recommended but others are also available.

1. Check your browser's version _(Chrome > About Google Chrome)_
![Browser Version](images/browser_version.png) 
<br>
2. Navigate to the [ChromeDriver downloads page](https://sites.google.com/a/chromium.org/chromedriver/downloads).
<br><br>
3. Download appropriately based on your browser's version and your OS.
![Download ChromeDriver zip file](images/chromedriver_options.png)

4. Unzip the driver.
<br><br>
5. Move to Applications folder (or wherever your Chrome application is).

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## Example 1 - YouTube

### Dynamic Pages

Some pages serve their content dynamically, which means they could look different each time they are loaded into the browser.  HTML that you see by inspecting elements in your browser might be missing from `requests` and `BeautifulSoup` because it is generated at access time.

In [3]:
query = "data science"
youtube_search = "https://www.youtube.com/results?search_query="
youtube_query = youtube_search + query.replace(' ', '+')

In [4]:
page = requests.get(youtube_query).text
soup = BeautifulSoup(page, 'html5lib')

ReadTimeout: HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=None)

In [ ]:
soup.find('div', id='contents')

Uh oh.  The video links should be under the contents div, but it's missing from our request.

> **QUESTION**: Why do you think this happened?

One option is to first load the page with Selenium THEN parse the page's HTML with BeautifulSoup.

First we launch the YouTube search page through our ChromeDrive.  A new browser should pop up.  **To continue using Selenium, keep this window open!**

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get(youtube_query)

We can access the page's HTML through the driver:

In [ ]:
driver.page_source[:1000]

Now we parse this with `BeautifulSoup` and the video information appears!

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
soup.find('div', id='contents')

In [ ]:
contents_div = soup.find('div', id='contents')

for title in contents_div.find_all('a', id='video-title'):
    print(title.text.strip())

> **QUESTION**: We only got about 20 video titles -- surely there are more videos about data science.  What do you think is happening?

### Interacting with Pages

We can also interact with pages using Selenium.  For example, we can 
- click
- type in input cells
- scroll
- drag and drop, etc.

If we want more data science video titles, we need to scroll down to the bottom of the screen for more videos to populate.

In [ ]:
for i in range(5):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    
    #Wait for page to load
    time.sleep(1)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
contents_div = soup.find('div', id='contents')

len(contents_div.find_all('a', id='video-title'))

Awesome!  Now we have several more videos to analyze and we could continue scrolling if we wanted even more.

What if we want to perform a new search for machine learning?

In [ ]:
search_box = driver.find_element_by_xpath("//input[@id='search']")

#clear the current search
search_box.clear()

#input new search
search_box.send_keys("machine learning")

#hit enter
search_box.send_keys(Keys.RETURN)  

time.sleep(1)

And can we filter to short videos (< 4 minutes) only?

In [ ]:
filter_button = driver.find_element_by_xpath(
    '//a[contains(@class, "ytd-toggle-button")]'
)
filter_button.click()

In [ ]:
short_link = driver.find_element_by_xpath(
    '//div[contains(@title, "Search for Short")]'
)
short_link.click()

Now we can either parse the page source with Beautiful Soup like before or pull text directly.  

For example, the title of the first short ML video (that isn't an ad!) can be found with:

In [ ]:
first_title = driver.find_element_by_xpath("//a[@id='video-title']")
first_title.text

In [ ]:
first_author = driver.find_element_by_xpath(
    "//ytd-video-renderer//ytd-channel-name//a"
)
first_author.text

#### Notes

- Check [here](https://www.w3schools.com/xml/xpath_syntax.asp) for additonal help writing xpath selectors.

- To select multiple elements, just switch to `driver.find_elements_by_xpath(...)`, which will return a list of matching elements.

- You can also access elements by id, name, etc.  Check [the docs](https://selenium-python.readthedocs.io/locating-elements.html) for more options.

Finally, when you are finished with the driver, be sure to close it.

In [ ]:
driver.close()

## Example 2 - Open Table  _(Optional)_

Let's try one more example: gathering information from Open Table about restaurants with available reservation slots.

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.opentable.com/')
time.sleep(1)  #pause to be sure page has loaded

Inspecting this page, we see the **name** of the drop down for picking the number of people is `Select_1`. Let's set the reservation for 4 people:

In [ ]:
people_dropdown = driver.find_element_by_xpath('//select[@aria-label="Party size selector"]')
people_dropdown.send_keys("4 people")
time.sleep(1)

Now select the reservation date: 3 days from now.

In [ ]:
from datetime import datetime, timedelta

In [ ]:
today = datetime.today()
today_truncated = datetime(today.year, today.month, today.day)
res_date = (today + timedelta(days=3)).strftime('%a, %b %d, %Y')
res_date

In [ ]:
#Expand the calendar
date_picker = driver.find_element_by_xpath('//div[@aria-label="Date selector"]')
date_picker.click()
time.sleep(1)

In [ ]:
#Select the date three days from now
date_element = driver.find_element_by_xpath(f'//div[@aria-label="{res_date}"]')
date_element.click()
time.sleep(1)

Set our reservation time for 8 PM.

In [ ]:
time_dropdown = driver.find_element_by_xpath('//select[@aria-label="Time selector"]')
time_dropdown.send_keys("8:00 PM")
time.sleep(1)

And search!

In [ ]:
search_button = driver.find_element_by_xpath("//*[contains(text(), 'Let’s go')]")
search_button.click()
time.sleep(1)

On this new page we find a long list of restaurants with available reservations for 4 people at roughly our desired day/time.  At this point we could grab the HTML (`driver.page_source`) and parse with BeautifulSoup.  

In [ ]:
soup = BeautifulSoup(driver.page_source)

In [ ]:
for rest in soup.find_all('div', class_='rest-row-header')[:20]:
    print(rest.find('a').text)

Or we could click into an individual restaurant to learn more.

In [ ]:
first_rest = driver.find_element_by_xpath('//div[@class="rest-row-header"]//a')
first_rest.click()

In [ ]:
rest_soup = BeautifulSoup(driver.page_source)

print(rest_soup.title.text)

> **QUESTION**:  Why doesn't the title of this page match up with this individual restaurant?

In [ ]:
#Switch windows!
driver.switch_to.window(driver.window_handles[1])

In [ ]:
rest_soup = BeautifulSoup(driver.page_source)

print(rest_soup.title.text)

As usual when working with Selenium, make sure to close your browser.  Since we have two windows up, we use `driver.quit()` to close the entire browser session.

In [ ]:
driver.quit()